In [1]:
from numpy.lib.twodim_base import diagflat
import pymongo
import csv
import pandas as pd
import numpy as np
from sklearn import svm, tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
dimension_reduced_ble = {}
with open('/Users/pdmcguckian/Documents/IoT/BLEData.csv', 'r') as file:
    count = 0
    date = 12
    bdrm = 0
    lvngrm = 0
    ktchn = 0
    real = False

    reader = csv.reader(file)
    for row in reader:
        if count == 1440:
            
            if date == 27:
                date = 28

            dimension_reduced_ble[date] = [timein, lvngrm, bdrm, ktchn]
            count = 0
            bdrm = 0
            lvngrm = 0
            ktchn = 0
            date += 1
            timein = 0
            real = False

        count +=1

        try:
            lvngrmRSSI = int(row[1])
        except:
            lvngrmRSSI = -100
        
        try:
            bdrmRSSI = int(row[2])
        except:
            bdrmRSSI = -100
        
        try:
            ktchnRSSI = int(row[3])
        except:
            ktchnRSSI = -100

        if (bdrmRSSI == -100 and lvngrmRSSI == -100 and ktchnRSSI == -100):
            pass

        elif (bdrmRSSI >= ktchnRSSI and bdrmRSSI >= lvngrmRSSI):
            bdrm+=1

            if not real:
                timein = count
            real = True

        elif (lvngrmRSSI >= bdrmRSSI and lvngrmRSSI >= ktchnRSSI):
            lvngrm += 1
            if not real:
                timein = count
            real = True

        elif (ktchnRSSI >= bdrmRSSI and ktchnRSSI >= lvngrmRSSI):
            ktchn+=1
            if not real:
                timein = count
            real = True
        pass

print(dimension_reduced_ble)

{12: [646, 4, 508, 22], 13: [0, 0, 0, 0], 14: [0, 0, 0, 0], 15: [468, 126, 582, 35], 16: [481, 26, 616, 87], 17: [753, 13, 569, 4], 18: [730, 0, 646, 8], 19: [0, 0, 0, 0], 20: [654, 75, 612, 8], 21: [464, 111, 660, 35], 22: [523, 8, 640, 40], 23: [497, 7, 688, 59], 24: [574, 17, 563, 2], 25: [525, 118, 581, 9], 26: [0, 0, 0, 0], 28: [0, 0, 0, 0], 29: [0, 0, 0, 0], 30: [634, 91, 473, 35], 31: [0, 0, 0, 0], 32: [695, 28, 554, 30], 33: [520, 21, 557, 73], 34: [398, 34, 738, 112], 35: [656, 69, 544, 22]}


In [3]:
dataset = []
with open('/Users/pdmcguckian/Documents/IoT/PredictionData.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        date = int(row[0])
        alcohol = int(row[1])
        sleep = int(row[2])

        if sleep != 0:
            if sleep > 84:
                sleep_quality = 1

            else:
                sleep_quality = 0

            instance = [alcohol, sleep_quality, dimension_reduced_ble[date][0], dimension_reduced_ble[date][1], dimension_reduced_ble[date][2], dimension_reduced_ble[date][3]]
            dataset.append(instance)

In [4]:
df = pd.DataFrame(dataset, columns=['Alcohol', 'Sleep', 'TimeIn', 'LivingRoom', 'Bedroom', 'Kitchen'])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df)

    Alcohol  Sleep  TimeIn  LivingRoom  Bedroom  Kitchen
0         0      1     646           4      508       22
1         1      1     468         126      582       35
2         0      1     523           8      640       40
3         0      1       0           0        0        0
4         0      0     753          13      569        4
5         0      0       0           0        0        0
6         1      1     398          34      738      112
7         1      0     520          21      557       73
8         0      0     525         118      581        9
9         0      1     481          26      616       87
10        1      1     497           7      688       59
11        1      0     656          69      544       22
12        0      0     730           0      646        8
13        1      0     634          91      473       35
14        1      1     464         111      660       35
15        0      0     574          17      563        2
16        0      0     695     

In [5]:
X = df.drop(columns=["Alcohol", "Sleep"])
y_s = df['Sleep']
y_a = df['Alcohol']

In [6]:
clf = svm.SVC(kernel='linear', C=10)
scores_s = cross_val_score(clf, X, y_s, cv=3)
scores_a = cross_val_score(clf, X, y_a, cv=3)

print("Sleep - SVM: %0.2f accuracy with a standard deviation of %0.2f" % (scores_s.mean(), scores_s.std())) 
print("Alcohol - SVM: %0.2f accuracy with a standard deviation of %0.2f" % (scores_a.mean(), scores_a.std())) 


Sleep - SVM: 0.61 accuracy with a standard deviation of 0.16
Alcohol - SVM: 0.78 accuracy with a standard deviation of 0.21


In [7]:
clf = tree.DecisionTreeClassifier(max_depth = 2, min_impurity_decrease=0.0)
scores_s = cross_val_score(clf, X, y_s, cv=3)
scores_a = cross_val_score(clf, X, y_a, cv=3)

print("Sleep - Decision Tree: %0.2f accuracy with a standard deviation of %0.2f" % (scores_s.mean(), scores_s.std()))
print("Alcohol - Decision Tree: %0.2f accuracy with a standard deviation of %0.2f" % (scores_a.mean(), scores_a.std()))

Sleep - Decision Tree: 0.61 accuracy with a standard deviation of 0.16
Alcohol - Decision Tree: 0.72 accuracy with a standard deviation of 0.16


In [8]:
clf = LogisticRegression(max_iter=1300, C=100)
scores_s = cross_val_score(clf, X, y_s, cv=3)
scores_a = cross_val_score(clf, X, y_a, cv=3)

print("Sleep - Logistic Regressioin: %0.2f accuracy with a standard deviation of %0.2f" % (scores_s.mean(), scores_s.std()))
print("Alcohol - Logistic Regressioin: %0.2f accuracy with a standard deviation of %0.2f" % (scores_a.mean(), scores_a.std()))

Sleep - Logistic Regressioin: 0.61 accuracy with a standard deviation of 0.16
Alcohol - Logistic Regressioin: 0.78 accuracy with a standard deviation of 0.21


In [9]:
poly = PolynomialFeatures(degree = 2, interaction_only=False, include_bias=False)
X = poly.fit_transform(X)
clf = LogisticRegression(max_iter=1300, C=100)
scores_s = cross_val_score(clf, X, y_s, cv=3)
scores_a = cross_val_score(clf, X, y_a, cv=3)

print("Sleep - Poly Logistic Regressioin: %0.2f accuracy with a standard deviation of %0.2f" % (scores_s.mean(), scores_s.std()))
print("Alcohol - Poly Logistic Regressioin: %0.2f accuracy with a standard deviation of %0.2f" % (scores_a.mean(), scores_a.std()))

Sleep - Poly Logistic Regressioin: 0.61 accuracy with a standard deviation of 0.21
Alcohol - Poly Logistic Regressioin: 0.56 accuracy with a standard deviation of 0.08
